# Sudoku solver

In [149]:
from math import ceil # for finding square, square index
from itertools import combinations # for block combinations
from functools import reduce # for ORing a list of lists

# If you want to animate the solving -- but this is hard on Windows
# from time import sleep # to show working step-by-step
# from pandas import DataFrame # pretty-print 2d lists as data frames

In [152]:
# input array: 0s represent empty squares to be filled
# puzzle = [
#     [2, 0, 0,   0, 1, 0,   0, 0, 5],
#     [0, 0, 1,   6, 8, 0,   0, 0, 0],
#     [0, 0, 5,   0, 3, 4,   0, 0, 6],
    
#     [0, 4, 0,   8, 0, 2,   5, 0, 0],
#     [6, 8, 0,   0, 0, 3,   0, 2, 0],
#     [3, 0, 2,   7, 0, 0,   0, 0, 0],
    
#     [9, 0, 0,   5, 2, 0,   6, 0, 0],
#     [0, 1, 0,   0, 7, 6,   9, 0, 0],
#     [5, 0, 0,   0, 9, 0,   0, 7, 8]]

# Try also (not solvable with pure simplification algorithm):
puzzle = [
    [8, 0, 0,   0, 0, 0,   0, 0, 0],
    [0, 0, 3,   6, 0, 0,   0, 0, 0],
    [0, 7, 0,   0, 9, 4,   2, 0, 6],
    
    [0, 5, 0,   0, 0, 7,   5, 0, 0],
    [0, 0, 0,   0, 4, 5,   7, 0, 0],
    [0, 0, 0,   1, 0, 0,   0, 3, 0],
    
    [0, 0, 1,   0, 0, 0,   0, 6, 8],
    [0, 0, 8,   5, 7, 6,   0, 1, 0],
    [0, 9, 0,   0, 0, 0,   4, 0, 0]]

In [153]:
# boring cleanup and administrative functions

# the "notes" for an entry are the set of possible values and label for its square, row, column
# e.g. notes for a "2" in block (5, 8) are: 
# [((6, 5), (5, 8), (8, 5)), False, True, False, False, False, False, False, False, False]

# generate label from co-ordinates
def to_label(i, j):
    square = 3 * (ceil(i / 3) - 1) + ceil(j / 3)
    square_pos = 3 * ((i - 1) % 3) + (j - 1) % 3  + 1 
    return ((square, square_pos), (i, j), (j, i))

# find row, column co-ordinates from square, square index co-ordinates
# interestingly, the formula is the same
def to_coord(p, q):
    row = 3 * (ceil(p / 3) - 1) + ceil(q / 3)
    col = 3 * ((p - 1) % 3) + (q - 1) % 3 + 1
    return (row, col)

# convert entry to notes
def entry_to_notes(entry, i, j):
    if entry == 0: return [to_label(i, j)] + [True for x in range(1, 10)]
    else: return [to_label(i, j)] + [True if x == entry else False for x in range(1, 10)]

# if "notes" is a singleton, pluck its element. Else 0.
def notes_to_entry(notes):
    if sum(notes[1:]) == 1: return notes.index(True)
    else: return 0

# transform readable puzzle into workable puzzle_notes
def to_puzzle_notes(puzzle_in): 
    return [[entry_to_notes(entry, i, j) for (j, entry) in enumerate(row, start = 1)] 
            for (i, row) in enumerate(puzzle_in, start = 1)]

# transform workable puzzle_notes into readable puzzle
def to_puzzle(puzzle_notes_in): return [[notes_to_entry(notes) for notes in row] for row in puzzle_notes_in]

# a simple function I'll need: ORing across arrays
def reduce_OR(list_of_lists):
    return reduce(lambda L1, L2 : [L1[x] or L2[x] for x in range(len(L1))], list_of_lists)[1:]

In [159]:
# SUDOKU SIMPLIFICATION ALGORITHM (typically solves puzzle completely):

## If the union of K blocks' notes in a row, column or square has cardinality K, we may 
## eliminate these K numbers from all other blocks in that row, column or square.

## We will check for the presence of such "K-plets" for each K and label these "eliminatable
## numbers" with the squares that they were found in (so we don't eliminate them from those 
## squares) and put them in a list.

# set_type: 0 (square), 1 (row), 2 (col); set_num: which row/col/square?
def K_plets(puzzle_notes_in, set_type, set_num, K):
    # Look for all notes whose label[set_type][0] == set_num
    set_contents = [note for row in puzzle_notes_in for note in row if note[0][set_type][0] == set_num]
    # consider their K-combinations
    set_combines = combinations(set_contents, K)
    # If the OR of a combination has K Trues, it's a K-plet
    plets = [combine for combine in set_combines if sum(reduce_OR(combine)) == K]
    # for each plet, create the object [[list of labels], [list of taken values]]
    plets_out = [([notes[0] for notes in combine], # list of labels
                  [val for val, bit in enumerate(reduce_OR(combine), start = 1) if bit == True]) # indices of 'True'
                 for combine in plets]
    return plets_out

# eliminate possibilities banned by K_plets; note has earlier discussed form
# [((6, 5), (5, 8), (8, 5)), False, True, False, False, False, False, False, False, False]
def update_set(puzzle_notes_in, set_type, set_num):
    for K in range(1, 9): # for loop faster here to avoid duplication of values in higher Ks, e.g. {2, 3}, {(2, 3)}
        for ban in K_plets(puzzle_notes_in, set_type, set_num, K): # for each K-plet
            puzzle_notes_in = [[[False if val in ban[1] else bit for val, bit in enumerate(notes)] # banned options to False
                             if notes[0][set_type][0] == set_num # for all blocks in the set
                             and notes[0] not in ban[0] # except those in the K-plet (and are thus the cause of the ban)
                             else notes for notes in row] for row in puzzle_notes_in] # leave everything else unchanged
    return puzzle_notes_in

# Sudoku simplification algorithm -- typically enough
def puzzle_simplify(puzzle_in):    
    puzzle_notes_in = to_puzzle_notes(puzzle_in) # initialise notes
    puzzle_notes_prev_in = [[10] * 9] * 9 # initialise "previous", for steady-state check
    while puzzle_notes_in != puzzle_notes_prev_in: # terminate upon reaching steady-state
        puzzle_notes_prev_in = puzzle_notes_in # update puzzle_prev
        for set_type in range(3): # do squares first, then rows, then columns
            for set_num in range(1, 10): # for each square/row/col
                puzzle_notes_in = update_set(puzzle_notes_in, set_type, set_num) # do some work
    return puzzle_notes_in

to_puzzle(puzzle_simplify(puzzle))

[[8, 0, 4, 7, 0, 0, 0, 0, 0],
 [9, 0, 3, 6, 0, 0, 0, 0, 0],
 [1, 7, 5, 0, 9, 4, 2, 0, 6],
 [0, 0, 0, 0, 0, 7, 0, 0, 0],
 [0, 8, 9, 0, 4, 5, 7, 2, 1],
 [0, 0, 0, 1, 0, 0, 0, 3, 0],
 [0, 0, 1, 4, 0, 9, 0, 6, 8],
 [0, 0, 8, 5, 7, 6, 9, 1, 0],
 [0, 9, 7, 0, 0, 0, 4, 0, 0]]

In [156]:
# Case-wise approach in case simplification did not fully solve it 

# breadth-first search